In [ ]:
# Function and variables replacement for C/C++ projects
import re  
p1 = '[a-z][a-z1-9]+([A-Z][a-z1-9]+)+' # camelCase，setThePath() 
p2 = '[A-Z][a-z1-9]+([A-Z][a-z1-9]+)+' # Pascal Case
p3 = '[a-z1-9]+([_][a-z1-9]+)+'       # set_vasible_value()
p4 = '[_][_]*(.+)'                    # start with __or_
p5 = '[a-z][_]([A-Z]*[a-z1-9]+)+'     # m_LpscStr(), also can be a variable
p6 = '([(](.+)[)])'                 # function with parameter,setBackgroundMode(SWT.INHERIT_DEFAULT)
p7 = '[a-z]+'
p8 = '[a-z][_](.+)'                 # e.g. m_pDoc，m_nCustomers
p9 = '(n|h|p|d|f|s|l|un|ch|sz|b|em|w|rc|dw|cr|dc)([A-Z][a-z]+)+' #  like, nStudentID  / hWndDlg; /  pPath / dValue
double_colon  = '::'
inclined_rod = '/'
dotM = '.'
bracketsM = '('
bracketsMM = '()'
function_name_list=[]
variable_name_list=[]
built_in_objects_names = ['dynamic_cast','reinterpret_cast','static_cast','wchar_t','const_cast','GitHub','HashMap']

fo = open(path_for_store,"w",encoding="utf-8")
for each_line in open(path_of_comments,'r'):
    y = each_line.split()
    for idx, i in enumerate(y):
        v1 = re.match(p1,i,flags=0)
        v2 = re.match(p2,i,flags=0)
        v3 = re.match(p3,i,flags=0)
        v4 = re.match(p4,i,flags=0)
        v5 = re.match(p5,i,flags=0)
        v7 = re.match(p7,i,flags=0)
        v8 = re.match(p8,i,flags=0)
        v9 = re.match(p9,i,flags=0)        
        i.replace(' ','')

        if i.find(inclined_rod) == -1 and i.find(bracketsMM) == -1 and i.find(double_colon) == -1:
            # there is a "."
            if dotM in i:
                i_lis = i.split('.')
                _yes_or_not = i_lis[1] in built_in_objects_names                
                # "." is not at the end
                if _yes_or_not == False and i_lis[1] != "":                    
                    v11 = re.match(p1,i_lis[1],flags=0)
                    v22 = re.match(p2,i_lis[1],flags=0)
                    v33 = re.match(p3,i_lis[1],flags=0) 
                    v44 = re.match(p4,i_lis[1],flags=0)
                    v88 = re.match(p8,i_lis[1],flags=0) 
                    v99 = re.match(p9,i_lis[1],flags=0)
                    if v33 != None or v11 != None or v44 != None or v22 != None:
                        # not all capital letter
                        if i_lis[1].isupper() == False:
                            function_name_list.append(i)
                            each_line = each_line.replace(i,'FUNCTION')
                    # variable 
                    if v88 != None or v99 != None:
                        variable_name_list.append(i)
                        each_line = each_line.replace(i,'VARIABLE')
                # "." is a period
                else:               
                    _yes_or_not = i_lis[0] in built_in_objects_names
                    if _yes_or_not == False:
                        if v1 != None or v2 != None or v4 != None:
                            if i.isupper() == False:
                                function_name_list.append(i)
                                each_line = each_line.replace(i,'FUNCTION')
                        # variable
                        if v8 != None or v9 != None:
                            variable_name_list.append(i)
                            each_line = each_line.replace(i,'VARIABLE')
            # no "."
            else:
                _yes_or_not = i in built_in_objects_names
                if _yes_or_not == False:
                    if v2 != None or v4 != None:
                         if i.isupper() == False:
                            function_name_list.append(i)
                            each_line = each_line.replace(i,'FUNCTION')
                    # variable
                    if v8 != None or v9 != None:
                        variable_name_list.append(i)
                        each_line = each_line.replace(i,'VARIABLE')
                        
                    # judge by preposition
                    if i == 'in'or i == 'for':
                        if y[idx+1] != "":
                            v33 = re.match(p3,y[idx+1],flags=0)
                            if v33 != None:
                                function_name_list.append(y[idx+1])
                                each_line = each_line.replace(y[idx+1],'FUNCTION')
        
        # followed by a (), like，setBackgroundMode() 
        if  i.find(double_colon) == -1 and "()" in i:
            function_name_list.append(i)
            each_line = each_line.replace(i,'FUNCTION')

        # function with parameter，setBackgroundMode(SWT.INHERIT_DEFAULT）
        if  i.find(double_colon) == -1 and bracketsM in i:
            split_list = i.split('(')
            if split_list[0] != "":               
                if v1 != None or v2 != None or v3 != None or v4 != None or v5 != None  or v7 != None:
                    ret6 = re.search(p6,'(' + split_list[1])
                    if ret6 != None:
                        function_name_list.append(i)
                        each_line = each_line.replace(i,'FUNCTION')

        # e.g. map “FeatureSpec::multipleMemories” to VARIABLE 
        # map “FeatureSpec::reset()” to FUNCTION. 
        if double_colon in i:
            # one or two ::
            _num_of_colon = i.count('::', 0, len(i))
            words_list = i.split('::')
            # one ::
            if _num_of_colon == 1 and words_list[1] != '':
                if bracketsM in words_list[1] or words_list[1] == 'FUNCTION':
                    function_name_list.append(i)
                    each_line = each_line.replace(i,'FUNCTION')

                # no ()
                else:
                    if words_list[1] != 'FUNCTION' and words_list[1] != 'VARIABLE':
                        variable_name_list.append(i)
                        each_line = each_line.replace(i,'VARIABLE')
            # two ::                 
            if _num_of_colon == 2 and words_list[2] != '':
                if bracketsM in words_list[2] or words_list[2] == 'FUNCTION':
                    function_name_list.append(i)
                    each_line = each_line.replace(i,'FUNCTION')
                else:
                    if words_list[2] != 'FUNCTION' and words_list[2] != 'VARIABLE':
                        variable_name_list.append(i)
                        each_line = each_line.replace(i,'VARIABLE')                       
    fo.write(each_line)           
fo.close()